In [1]:
import numpy as np
import pandas as pd
import scipy
from cuml import KalmanFilter
import cudf
import os

# Helper Functions

In [2]:
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [3]:
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz',source='mortgage'):
    if os.path.exists(cached) and source=='mortgage':
        print('use mortgage data')
        with gzip.open(cached) as f:
            X = np.load(f)
        X = X[np.random.randint(0,X.shape[0]-1,nrows),:ncols]
    else:
        print('use random data')
        X = np.random.random((nrows,ncols)).astype('float32')
    df = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])}).fillna(0)
    return df

In [4]:
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=1e-2,with_sign=True,metric='mse'):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    if metric=='mse':
        error = mean_squared_error(a,b)
    else:
        error = np.sum(a!=b)/(a.shape[0]*a.shape[1])
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x    

In [5]:
def spKalman(data, x, z, n_iter = 50):
    # intial parameters
    sz = data # size of array
    x = -0.37727 # truth value (typo in example at top of p. 13 calls this z)
    z = np.random.normal(x,0.1,size=sz) # observations (normal about x, sigma=0.1)

    Q = 1e-5 # process variance

    # allocate space for arrays
    xhat=np.zeros(sz)      # a posteri estimate of x
    P=np.zeros(sz)         # a posteri error estimate
    xhatminus=np.zeros(sz) # a priori estimate of x
    Pminus=np.zeros(sz)    # a priori error estimate
    K=np.zeros(sz)         # gain or blending factor

    R = 0.1**2 # estimate of measurement variance, change to see effect

    # intial guesses
    xhat[0] = 0.0
    P[0] = 1.0

    for k in range(1,n_iter):
        # time update
        xhatminus[k] = xhat[k-1]
        Pminus[k] = P[k-1]+Q

        # measurement update
        K[k] = Pminus[k]/( Pminus[k]+R )
        xhat[k] = xhatminus[k]+K[k]*(z[k]-xhatminus[k])
        P[k] = (1-K[k])*Pminus[k]
    return Pminus 

In [10]:
def cuMLKalman(data, dim_x,dim_z, n_iter = 50):
    f = KalmanFilter(dim_x, dim_z)
    f.x = np.array(data)   # velocity
    f.F = np.array([[1.,1.], [0.,1.]])
    f.H = np.array([[1.,0.]])
    f.P = np.array([[1000., 0.], [   0., 1000.] ])
    f.R = 5

    for k in range(1,n_iter):
        z = numba.cuda.to_device(np.array([i]))
        f.predict()
        f.update(z)

# Run tests

In [11]:
%%time
nrows = 2**16
ncols = 40
n_iter = 50 

X = load_data(nrows,ncols)

#spKalman(X, nrows, ncols, n_iter)
#cuMLKalman(X, nrows, ncols, n_iter)

print('data',X.shape)

use random data
data (65536, 40)
CPU times: user 52.2 ms, sys: 21.2 ms, total: 73.4 ms
Wall time: 71.3 ms


In [13]:
import numpy as np
#import matplotlib.pyplot as plt

#plt.rcParams['figure.figsize'] = (10, 8)

# intial parameters
n_iter = 50
sz = (n_iter,) # size of array
x = -0.37727 # truth value (typo in example at top of p. 13 calls this z)
z = np.random.normal(x,0.1,size=sz) # observations (normal about x, sigma=0.1)

Q = 1e-5 # process variance

# allocate space for arrays
xhat=np.zeros(sz)      # a posteri estimate of x
P=np.zeros(sz)         # a posteri error estimate
xhatminus=np.zeros(sz) # a priori estimate of x
Pminus=np.zeros(sz)    # a priori error estimate
K=np.zeros(sz)         # gain or blending factor

R = 0.1**2 # estimate of measurement variance, change to see effect

# intial guesses
xhat[0] = 0.0
P[0] = 1.0

for k in range(1,n_iter):
    # time update
    xhatminus[k] = xhat[k-1]
    Pminus[k] = P[k-1]+Q

    # measurement update
    K[k] = Pminus[k]/( Pminus[k]+R )
    xhat[k] = xhatminus[k]+K[k]*(z[k]-xhatminus[k])
    P[k] = (1-K[k])*Pminus[k]


In [18]:
P

array([1.00000000e+00, 9.90099108e-03, 4.97764829e-03, 3.32783916e-03,
       2.50253367e-03, 2.00801352e-03, 1.67915730e-03, 1.44506337e-03,
       1.27023598e-03, 1.13493723e-03, 1.02731600e-03, 9.39826313e-04,
       8.67435050e-04, 8.06656207e-04, 7.54998764e-04, 7.10635255e-04,
       6.72194546e-04, 6.38627712e-04, 6.09118592e-04, 5.83022580e-04,
       5.59823767e-04, 5.39104321e-04, 5.20522221e-04, 5.03794788e-04,
       4.88686339e-04, 4.74998798e-04, 4.62564476e-04, 4.51240455e-04,
       4.40904171e-04, 4.31449915e-04, 4.22786029e-04, 4.14832651e-04,
       4.07519876e-04, 4.00786254e-04, 3.94577550e-04, 3.88845725e-04,
       3.83548074e-04, 3.78646514e-04, 3.74106976e-04, 3.69898902e-04,
       3.65994799e-04, 3.62369880e-04, 3.59001737e-04, 3.55870070e-04,
       3.52956449e-04, 3.50244113e-04, 3.47717785e-04, 3.45363518e-04,
       3.43168559e-04, 3.41121230e-04])

In [39]:
import pytest
import cudf
import numpy as np
import pdb
from numba import cuda
from math import sqrt
from cuml import KalmanFilter

def test_linear_kalman_filter(precision, dim_x, dim_z, input_type):
    f = KalmanFilter(dim_x=dim_x, dim_z=dim_z, precision=precision)

    if precision == 'single':
        dt = np.float32
    else:
        dt = np.float64

    if input_type == 'numpy':

        f.x = np.array([1.0,2.0,3.0,4.0,5.0,6.0,7.0], dtype=dt)
        pdb.set_trace()
        f.F = np.eye(dim_x, dtype=dt)

        h = np.zeros((dim_x, dim_z), dtype=dt)
        h[0] = 1

        f.H = h
        f.P = np.eye(dim_x, dtype=dt)*1000
        f.R = np.eye(dim_z, dtype=dt)*5.0

    else:

        f.x = np_to_dataframe(np.zeros((dim_x, 1), dtype=dt))

        tmp = np.eye(dim_x, dtype=dt, order='F')

        f.F = np_to_dataframe(tmp)

        h = np.zeros((dim_x, dim_z), dtype=dt, order='F')
        h[0] = 1

        f.H = np_to_dataframe(h)
        f.P = np_to_dataframe(np.eye(dim_x, dtype=dt, order='F')*1000)
        f.R = np_to_dataframe(np.eye(dim_z, dtype=dt, order='F')*5.0)

    rmse_x = 0
    rmse_v = 0

    n = 10

    for i in range(n):
        f.predict()

        z = i*np.ones(dim_z, dtype=dt)

        f.update(cuda.to_device(np.array(z, dtype=dt)))
        pdb.set_trace()
        x = f.x.copy_to_host()
        rmse_x = rmse_x + ((x[0] - i)**2)
        rmse_v = rmse_v + ((x[1] - 1)**2)

    return f

In [40]:
info = test_linear_kalman_filter('single', 7, 5, 'numpy')

> <ipython-input-39-5d116cd2af0c>(21)test_linear_kalman_filter()
-> f.F = np.eye(dim_x, dtype=dt)


(Pdb)  np.asarray(f.x)


array([1., 2., 3., 4., 5., 6., 7.])


(Pdb)  c


> <ipython-input-39-5d116cd2af0c>(57)test_linear_kalman_filter()
-> x = f.x.copy_to_host()


...

> <ipython-input-39-5d116cd2af0c>(56)test_linear_kalman_filter()
-> pdb.set_trace()


(Pdb)  np.asarray(f.x)


array([2.99898219, 2.        , 3.        , 4.        , 5.        ,
       6.        , 7.        ])


(Pdb)  exit


BdbQuit: 

In [ ]:
info = test_linear_kalman_filter('single', 7, 1, 'numpy')

> <ipython-input-39-5d116cd2af0c>(21)test_linear_kalman_filter()
-> f.F = np.eye(dim_x, dtype=dt)


(Pdb)  c


> <ipython-input-39-5d116cd2af0c>(57)test_linear_kalman_filter()
-> x = f.x.copy_to_host()


(Pdb)  np.asarray(info.x)


array([[8.99903965],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ]])


(Pdb)   np.asarray(info.z)


array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])


(Pdb)   np.asarray(info.R)


array([[5., 0., 0., 0., 0.],
       [0., 5., 0., 0., 0.],
       [0., 0., 5., 0., 0.],
       [0., 0., 0., 5., 0.],
       [0., 0., 0., 0., 5.]])


In [31]:
np.asarray(info.P)

array([[9.99011755e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 5.12512000e+05, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 5.12512000e+05, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.12512000e+05,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        5.12512000e+05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
   

In [ ]:
n_neighbors = 10

In [ ]:
%%time
knn_sk = skKNN(X)
D_sk,I_sk = knn_sk.query(X,n_neighbors)

In [ ]:
%%time
X = cudf.DataFrame.from_pandas(X)

In [ ]:
%%time
knn_cuml = cumlKNN(n_gpus=1)
knn_cuml.fit(X)
D_cuml,I_cuml = knn_cuml.query(X,n_neighbors)

In [ ]:
passed = array_equal(D_sk,D_cuml)
message = 'compare knn: cuml vs sklearn distances %s'%('equal'if passed else 'NOT equal')
print(message)
passed = array_equal(I_sk,I_cuml)
message = 'compare knn: cuml vs sklearn indexes %s'%('equal'if passed else 'NOT equal')
print(message)